# Sea Ice Diagnostics for two CESM3 runs

In [ ]:
import xarray as xr
import numpy as np
import yaml
import os

import cupid_utils as cu

In [ ]:
# These parameters are needed when running interactively

CESM_output_dir = ""  # "/glade/campaign/cesm/development/cross-wg/diagnostic_framework/CESM_output_for_testing"
ts_dir = None  # "/glade/campaign/cesm/development/cross-wg/diagnostic_framework/CESM_output_for_testing"

case_name = ""  # "b.e30_beta04.BLT1850.ne30_t232_wgx3.121congel"
base_case_name = ""  # "b.e30_beta04.BLT1850.ne30_t232_wgx3.121"

start_date = ""  # "0001-01-01"
end_date = ""  # "0100-01-01"
base_start_date = ""  # "0001-01-01"
base_end_date = ""  # "0100-01-01"

obs_data_dir = ""  # "/glade/campaign/cesm/development/cross-wg/diagnostic_framework/CUPiD_obs_data"
grid_file = ""  # "/glade/campaign/cesm/community/omwg/grids/tx2_3v2_grid.nc"
climo_nyears = 35

serial = False  # use dask LocalCluster

lc_kwargs = {}

In [ ]:
# Set some parameter defaults
if ts_dir is None:
    ts_dir = CESM_output_dir

In [ ]:
# When running interactively, cupid_run should be set to 0 for
# a DASK cluster

cupid_run = 1

if cupid_run == 1:

    from dask.distributed import Client, LocalCluster

    # Spin up cluster (if running in parallel)
    client = None
    if not serial:
        cluster = LocalCluster(**lc_kwargs)
        client = Client(cluster)

else:

    from dask.distributed import Client
    from dask_jobqueue import PBSCluster

    cluster = PBSCluster(
        cores=16,
        processes=16,
        memory="100GB",
        account="P93300065",
        queue="casper",
        walltime="02:00:00",
    )

    client = Client(cluster)

    cluster.scale(1)

    print(cluster)
print(client)

In [ ]:
# Read in two cases. The ADF timeseries are needed here.

ds1 = xr.open_mfdataset(
    CESM_output_dir
    + "/"
    + case_name
    + "/ice/proc/tseries/"
    + case_name
    + ".cice.h.*.nc",
    data_vars="minimal",
    compat="override",
    coords="minimal",
).sel(time=slice(start_date, end_date))
ds2 = xr.open_mfdataset(
    ts_dir
    + "/"
    + base_case_name
    + "/ice/proc/tseries/"
    + base_case_name
    + ".cice.h.*.nc",
    data_vars="minimal",
    compat="override",
    coords="minimal",
).sel(time=slice(base_start_date, base_end_date))

ds_grid = xr.open_dataset(grid_file)
TLAT = ds_grid["TLAT"]
TLON = ds_grid["TLONG"]
tarea = ds_grid["TAREA"] * 1.0e-4
angle = ds_grid["ANGLE"]

ds1_ann = ds1.resample(time="YS").mean(dim="time")
ds2_ann = ds2.resample(time="YS").mean(dim="time")

climo_nyears = min(climo_nyears, len(ds1_ann.time))
climo_nyears = min(climo_nyears, len(ds2_ann.time))

yml_dir = os.path.join(cu.__path__[0], "..", "input_files", "ice")
with open(os.path.join(yml_dir, "cice_masks.yml"), "r") as file:
    cice_masks = yaml.safe_load(file)

with open(os.path.join(yml_dir, "cice_vars.yml"), "r") as file:
    cice_vars = yaml.safe_load(file)

path_HadleyOI = os.path.join(obs_data_dir, "ice", "analysis_datasets", "1x1d/")

In [ ]:
from cupid_utils.ice import plot_diff
from cupid_utils.ice import vect_diff

In [ ]:
for var in cice_vars:
    vmin = cice_vars[var][0]["levels"][0]
    vmax = cice_vars[var][0]["levels"][-1]
    levels = np.array(cice_vars[var][0]["levels"])
    title = cice_vars[var][1]["title"]
    if var in ds1:
        field1 = ds1[var].isel(time=slice(-climo_nyears * 12, None))
    else:
        field1 = ds1["aice"].isel(time=slice(-climo_nyears * 12, None)) * 0.0
    if var in ds2:
        field2 = ds2[var].isel(time=slice(-climo_nyears * 12, None))
    else:
        field2 = ds2["aice"].isel(time=slice(-climo_nyears * 12, None)) * 0.0
    plot_diff(
        field1,
        field2,
        levels,
        case_name,
        base_case_name,
        title,
        "N",
        TLAT,
        TLON,
        path_HadleyOI,
    )

In [ ]:
for var in cice_vars:
    vmin = cice_vars[var][0]["levels"][0]
    vmax = cice_vars[var][0]["levels"][1]
    levels = np.array(cice_vars[var][0]["levels"])
    title = cice_vars[var][1]["title"]
    if var in ds1:
        field1 = ds1[var].isel(time=slice(-climo_nyears * 12, None))
    else:
        field1 = ds1["aice"].isel(time=slice(-climo_nyears * 12, None)) * 0.0
    if var in ds2:
        field2 = ds2[var].isel(time=slice(-climo_nyears * 12, None))
    else:
        field2 = ds2["aice"].isel(time=slice(-climo_nyears * 12, None)) * 0.0
    plot_diff(
        field1,
        field2,
        levels,
        case_name,
        base_case_name,
        title,
        "S",
        TLAT,
        TLON,
        path_HadleyOI,
    )

In [ ]:
if "uvel" in ds1_ann.variables:
    uvel1 = ds1_ann["uvel"].isel(time=slice(-climo_nyears, None)).mean("time").squeeze()
    vvel1 = ds1_ann["vvel"].isel(time=slice(-climo_nyears, None)).mean("time").squeeze()
    uvel2 = ds2_ann["uvel"].isel(time=slice(-climo_nyears, None)).mean("time").squeeze()
    vvel2 = ds2_ann["vvel"].isel(time=slice(-climo_nyears, None)).mean("time").squeeze()

    vect_diff(
        uvel1.values,
        vvel1.values,
        uvel2.values,
        vvel2.values,
        angle.values,
        "N",
        case_name,
        base_case_name,
        TLAT.values,
        TLON.values,
    )

In [ ]:
if "uvel" in ds1_ann.variables:
    vect_diff(
        uvel1.values,
        vvel1.values,
        uvel2.values,
        vvel2.values,
        angle.values,
        "S",
        case_name,
        base_case_name,
        TLAT.values,
        TLON.values,
    )

In [ ]:
if client and not cupid_run:
    client.shutdown()